# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0829 18:01:22.157000 4114138 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0829 18:01:22.157000 4114138 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-29 18:01:22] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=34020, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=89443

[2025-08-29 18:01:23] Using default HuggingFace chat template with detected content format: string


W0829 18:01:30.392000 4115252 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0829 18:01:30.392000 4115252 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0829 18:01:30.487000 4115251 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0829 18:01:30.487000 4115251 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-29 18:01:31] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-29 18:01:31] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=37.19 GB): 100%|██████████| 3/3 [00:00<00:00, 10.93it/s]
[2025-08-29 18:01:37] Capture cuda graph end. Time elapsed: 0.79 s. mem usage=0.12 GB. avail mem=37.14 GB.


[2025-08-29 18:01:38] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=37.12 GB


[2025-08-29 18:01:38] INFO:     Started server process [4114138]
[2025-08-29 18:01:38] INFO:     Waiting for application startup.
[2025-08-29 18:01:38] INFO:     Application startup complete.
[2025-08-29 18:01:38] INFO:     Uvicorn running on http://0.0.0.0:34020 (Press CTRL+C to quit)


[2025-08-29 18:01:39] INFO:     127.0.0.1:44470 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-29 18:01:39] INFO:     127.0.0.1:44476 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-29 18:01:39] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:40] INFO:     127.0.0.1:44490 - "POST /generate HTTP/1.1" 200 OK
[2025-08-29 18:01:40] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-29 18:01:44] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:46] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 4.51, #queue-req: 0, 


[2025-08-29 18:01:47] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 162.43, #queue-req: 0, 


[2025-08-29 18:01:47] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.14, #queue-req: 0, 


[2025-08-29 18:01:47] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.01, #queue-req: 0, 


...

[2025-08-29 18:01:48] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 142.58, #queue-req: 0, 


[2025-08-29 18:01:48] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 139.24, #queue-req: 0, 


[2025-08-29 18:01:49] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 145.88, #queue-req: 0, 


[2025-08-29 18:01:49] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 149.77, #queue-req: 0, 


[2025-08-29 18:01:49] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 130.24, #queue-req: 0, 
[2025-08-29 18:01:49] INFO:     127.0.0.1:52134 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-29 18:01:49] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:50] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 94.10, #queue-req: 0, 


[2025-08-29 18:01:50] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 82.88, #queue-req: 0, 


[2025-08-29 18:01:50] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 146.37, #queue-req: 0, 


[2025-08-29 18:01:51] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.91, #queue-req: 0, 


...

[2025-08-29 18:01:51] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 135.11, #queue-req: 0, 


[2025-08-29 18:01:52] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 138.37, #queue-req: 0, 


[2025-08-29 18:01:52] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 144.84, #queue-req: 0, 


[2025-08-29 18:01:52] Decode batch. #running-req: 1, #token: 416, token usage: 0.02, cuda graph: True, gen throughput (token/s): 147.09, #queue-req: 0, 


[2025-08-29 18:01:52] Decode batch. #running-req: 1, #token: 456, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.22, #queue-req: 0, 
[2025-08-29 18:01:52] INFO:     127.0.0.1:52134 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-29 18:01:52] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:53] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: True, gen throughput (token/s): 144.77, #queue-req: 0, 


[2025-08-29 18:01:53] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, cuda graph: True, gen throughput (token/s): 156.94, #queue-req: 0, 


[2025-08-29 18:01:53] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.99, #queue-req: 0, 


[2025-08-29 18:01:53] Decode batch. #running-req: 1, #token: 184, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.29, #queue-req: 0, 


...

[2025-08-29 18:01:56] Decode batch. #running-req: 1, #token: 584, token usage: 0.03, cuda graph: True, gen throughput (token/s): 166.62, #queue-req: 0, 


[2025-08-29 18:01:56] Decode batch. #running-req: 1, #token: 624, token usage: 0.03, cuda graph: True, gen throughput (token/s): 147.54, #queue-req: 0, 


[2025-08-29 18:01:56] Decode batch. #running-req: 1, #token: 664, token usage: 0.03, cuda graph: True, gen throughput (token/s): 167.26, #queue-req: 0, 


[2025-08-29 18:01:57] Decode batch. #running-req: 1, #token: 704, token usage: 0.03, cuda graph: True, gen throughput (token/s): 166.58, #queue-req: 0, 


[2025-08-29 18:01:57] Decode batch. #running-req: 1, #token: 744, token usage: 0.04, cuda graph: True, gen throughput (token/s): 163.69, #queue-req: 0, 
[2025-08-29 18:01:57] INFO:     127.0.0.1:52134 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-29 18:01:57] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:57] Decode batch. #running-req: 1, #token: 34, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.03, #queue-req: 0, 


[2025-08-29 18:01:57] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.25, #queue-req: 0, 


[2025-08-29 18:01:58] Decode batch. #running-req: 1, #token: 114, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.50, #queue-req: 0, 
[2025-08-29 18:01:58] INFO:     127.0.0.1:52134 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-29 18:01:58] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:01:58] Decode batch. #running-req: 1, #token: 494, token usage: 0.02, cuda graph: True, gen throughput (token/s): 114.22, #queue-req: 0, 


[2025-08-29 18:01:58] Decode batch. #running-req: 1, #token: 534, token usage: 0.03, cuda graph: True, gen throughput (token/s): 161.28, #queue-req: 0, 


[2025-08-29 18:01:58] Decode batch. #running-req: 1, #token: 574, token usage: 0.03, cuda graph: True, gen throughput (token/s): 153.63, #queue-req: 0, 


[2025-08-29 18:01:59] Decode batch. #running-req: 1, #token: 614, token usage: 0.03, cuda graph: True, gen throughput (token/s): 115.19, #queue-req: 0, 


[2025-08-29 18:01:59] Decode batch. #running-req: 1, #token: 654, token usage: 0.03, cuda graph: True, gen throughput (token/s): 119.36, #queue-req: 0, 


[2025-08-29 18:01:59] Decode batch. #running-req: 1, #token: 694, token usage: 0.03, cuda graph: True, gen throughput (token/s): 164.97, #queue-req: 0, 


[2025-08-29 18:02:00] Decode batch. #running-req: 1, #token: 734, token usage: 0.04, cuda graph: True, gen throughput (token/s): 164.80, #queue-req: 0, 


[2025-08-29 18:02:00] Decode batch. #running-req: 1, #token: 774, token usage: 0.04, cuda graph: True, gen throughput (token/s): 159.43, #queue-req: 0, 


[2025-08-29 18:02:00] Decode batch. #running-req: 1, #token: 814, token usage: 0.04, cuda graph: True, gen throughput (token/s): 161.21, #queue-req: 0, 
[2025-08-29 18:02:00] INFO:     127.0.0.1:52134 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-29 18:02:02] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-29 18:02:02] Decode batch. #running-req: 1, #token: 43, token usage: 0.00, cuda graph: True, gen throughput (token/s): 22.11, #queue-req: 0, 


[2025-08-29 18:02:02] Decode batch. #running-req: 1, #token: 83, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.21, #queue-req: 0, 


[2025-08-29 18:02:02] Decode batch. #running-req: 1, #token: 123, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.10, #queue-req: 0, 


[2025-08-29 18:02:03] Decode batch. #running-req: 1, #token: 163, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.78, #queue-req: 0, 


[2025-08-29 18:02:03] Decode batch. #running-req: 1, #token: 203, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.50, #queue-req: 0, 


[2025-08-29 18:02:03] Decode batch. #running-req: 1, #token: 243, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.41, #queue-req: 0, 


[2025-08-29 18:02:03] Decode batch. #running-req: 1, #token: 283, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.32, #queue-req: 0, 


[2025-08-29 18:02:04] Decode batch. #running-req: 1, #token: 323, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.44, #queue-req: 0, 


[2025-08-29 18:02:04] Decode batch. #running-req: 1, #token: 363, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.30, #queue-req: 0, 


[2025-08-29 18:02:04] Decode batch. #running-req: 1, #token: 403, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.14, #queue-req: 0, 
[2025-08-29 18:02:04] INFO:     127.0.0.1:56450 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-29 18:02:04] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:02:04] Decode batch. #running-req: 1, #token: 38, token usage: 0.00, cuda graph: True, gen throughput (token/s): 154.13, #queue-req: 0, 


[2025-08-29 18:02:05] Decode batch. #running-req: 1, #token: 78, token usage: 0.00, cuda graph: True, gen throughput (token/s): 166.25, #queue-req: 0, 


[2025-08-29 18:02:05] Decode batch. #running-req: 1, #token: 118, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.12, #queue-req: 0, 


[2025-08-29 18:02:05] Decode batch. #running-req: 1, #token: 158, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.57, #queue-req: 0, 


...

[2025-08-29 18:02:06] Decode batch. #running-req: 1, #token: 238, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.43, #queue-req: 0, 


[2025-08-29 18:02:06] Decode batch. #running-req: 1, #token: 278, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.30, #queue-req: 0, 


[2025-08-29 18:02:06] Decode batch. #running-req: 1, #token: 318, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.28, #queue-req: 0, 


[2025-08-29 18:02:06] Decode batch. #running-req: 1, #token: 358, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.20, #queue-req: 0, 


[2025-08-29 18:02:07] Decode batch. #running-req: 1, #token: 398, token usage: 0.02, cuda graph: True, gen throughput (token/s): 164.19, #queue-req: 0, 
[2025-08-29 18:02:07] INFO:     127.0.0.1:56456 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-29 18:02:07] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-29 18:02:07] Prefill batch. #new-seq: 3, #new-token: 3, #cached-token: 30, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:02:07] Decode batch. #running-req: 3, #token: 98, token usage: 0.00, cuda graph: True, gen throughput (token/s): 200.76, #queue-req: 0, 


[2025-08-29 18:02:07] Decode batch. #running-req: 3, #token: 218, token usage: 0.01, cuda graph: True, gen throughput (token/s): 484.66, #queue-req: 0, 


[2025-08-29 18:02:08] Decode batch. #running-req: 3, #token: 338, token usage: 0.02, cuda graph: True, gen throughput (token/s): 484.53, #queue-req: 0, 


[2025-08-29 18:02:08] Decode batch. #running-req: 3, #token: 458, token usage: 0.02, cuda graph: True, gen throughput (token/s): 479.99, #queue-req: 0, 


[2025-08-29 18:02:08] Decode batch. #running-req: 3, #token: 578, token usage: 0.03, cuda graph: True, gen throughput (token/s): 478.62, #queue-req: 0, 


[2025-08-29 18:02:08] Decode batch. #running-req: 3, #token: 698, token usage: 0.03, cuda graph: True, gen throughput (token/s): 478.58, #queue-req: 0, 
[2025-08-29 18:02:08] INFO:     127.0.0.1:56468 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nThe capital of France is Paris. It is located in the northern part of the country, along the Seine River. Paris is known for its rich history, art, and landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Damme Cathedral. It is a major city in France and has a significant cultural and economic impact.\n\nPlease provide the information in a clear and concise manner, using bullet points for the location and key landmarks

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-29 18:02:08] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-29 18:02:09] Decode batch. #running-req: 1, #token: 21, token usage: 0.00, cuda graph: True, gen throughput (token/s): 347.12, #queue-req: 0, 


[2025-08-29 18:02:09] Decode batch. #running-req: 1, #token: 61, token usage: 0.00, cuda graph: True, gen throughput (token/s): 156.55, #queue-req: 0, 


[2025-08-29 18:02:09] Decode batch. #running-req: 1, #token: 101, token usage: 0.00, cuda graph: True, gen throughput (token/s): 169.09, #queue-req: 0, 


[2025-08-29 18:02:09] Decode batch. #running-req: 1, #token: 141, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.60, #queue-req: 0, 


...

[2025-08-29 18:02:20] Decode batch. #running-req: 1, #token: 1901, token usage: 0.09, cuda graph: True, gen throughput (token/s): 164.35, #queue-req: 0, 


[2025-08-29 18:02:20] Decode batch. #running-req: 1, #token: 1941, token usage: 0.09, cuda graph: True, gen throughput (token/s): 165.68, #queue-req: 0, 


[2025-08-29 18:02:21] Decode batch. #running-req: 1, #token: 1981, token usage: 0.10, cuda graph: True, gen throughput (token/s): 166.47, #queue-req: 0, 


[2025-08-29 18:02:21] Decode batch. #running-req: 1, #token: 2021, token usage: 0.10, cuda graph: True, gen throughput (token/s): 160.94, #queue-req: 0, 


[2025-08-29 18:02:21] INFO:     127.0.0.1:56482 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-29 18:02:21] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-29 18:02:21] Decode batch. #running-req: 1, #token: 30, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.48, #queue-req: 0, 


[2025-08-29 18:02:21] Decode batch. #running-req: 1, #token: 70, token usage: 0.00, cuda graph: True, gen throughput (token/s): 164.14, #queue-req: 0, 


[2025-08-29 18:02:22] Decode batch. #running-req: 1, #token: 110, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.44, #queue-req: 0, 


[2025-08-29 18:02:22] Decode batch. #running-req: 1, #token: 150, token usage: 0.01, cuda graph: True, gen throughput (token/s): 163.70, #queue-req: 0, 


[2025-08-29 18:02:22] Decode batch. #running-req: 1, #token: 190, token usage: 0.01, cuda graph: True, gen throughput (token/s): 162.20, #queue-req: 0, 


...

[2025-08-29 18:02:25] Decode batch. #running-req: 1, #token: 670, token usage: 0.03, cuda graph: True, gen throughput (token/s): 126.37, #queue-req: 0, 


[2025-08-29 18:02:25] Decode batch. #running-req: 1, #token: 710, token usage: 0.03, cuda graph: True, gen throughput (token/s): 152.62, #queue-req: 0, 


[2025-08-29 18:02:26] Decode batch. #running-req: 1, #token: 750, token usage: 0.04, cuda graph: True, gen throughput (token/s): 164.79, #queue-req: 0, 


[2025-08-29 18:02:26] Decode batch. #running-req: 1, #token: 790, token usage: 0.04, cuda graph: True, gen throughput (token/s): 162.45, #queue-req: 0, 


[2025-08-29 18:02:26] Decode batch. #running-req: 1, #token: 830, token usage: 0.04, cuda graph: True, gen throughput (token/s): 161.38, #queue-req: 0, 
[2025-08-29 18:02:26] INFO:     127.0.0.1:45244 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0829 18:02:28.347000 4113252 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0829 18:02:28.347000 4113252 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0829 18:02:36.160000 4120828 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0829 18:02:36.160000 4120828 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.19s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.15s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.15s/it]



Capturing batches (bs=4 avail_mem=8.92 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=8.85 GB): 100%|██████████| 3/3 [00:00<00:00,  5.61it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugoslav",
    "Other"
  ],
  "KeySights": [
    "The Great Wall",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need any other information!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude": 48.8566,
    "longitude": 

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. Here are some key facts about Paris:

1. **Historical Significance**: Paris has been the capital of France since the early Middle Ages. It holds significant historical importance, especially due to its role as the birthplace of the French Revolution.

2. **Cultural Landmarks**: Paris is renowned for its rich cultural heritage, featuring iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.

3. **Economic Importance**: As the economic capital of France, Paris plays a crucial role in the national and European economies. It is a hub for finance
Prompt: Give me the information of the capital of Germany.
Generated text: 
Okay, so I need to figure out the capital of Germany. Hmm, I know that Germany is a country in Europe, right? And I've heard that Berlin is a major city there. But is it the capital? I'm pretty sure it's the capital, but I'm not 100% c

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its history, current issues, and future.

Please structure your response with clear headings and bullet points for clarity.

Certainly! Here's a structured overview of London as a major global city:

**London: A Major Global City**

- **Historical Significance**
  - **Ancient Beginnings**: London's roots predate the British Empire, with origins in the Thames Valley.
  - **Roman Influence**: Became a significant settlement, home to Londinium, a key Roman town.
  - **Medieval Growth**: The Domesday Book records it as a thriving area in 1086, evolving into a bustling urban center
Prompt: Please provide information about Paris as a major global city:
Generated text:  its history, culture, and major attractions.

6.1.2.1

Please respond to the above query in detail, using proper English, and structure your response with the following headers: "History", "Culture", "Major Attractions", and "Additional Po

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Okay, so the user asked for the information and population of the capital of France in JSON format. Let me break this down.

First, I need to identify the capital of France. I know that Paris is the capital. That's straightforward.

Next, I have to find out the current population of Paris. I remember that the population can change over time due to various factors like births, deaths, and migration. I think the latest estimate might be around 2 million people. But I should double-check that to be accurate.

Now, the user wants this information in JSON format. JSON stands for JavaScript Object Notation, and it's a way to structure data. I need to create a JSON object with the key-value pairs for the city, population, and maybe add some sub-keys for better detail. Perhaps I can include the population range to 

In [19]:
llm.shutdown()